# Construisez un modèle de scoring

# Prêt à dépenser

<img src="logo_pret_a_depenser.png" alt="logo_pret_a_depenser" width="300"/>

# Problématique de l'entreprise

"Prêt à dépenser" propose des crédits à la consommation pour les particuliers. La cible commerciale de l'entreprise sont les personnes n'ayant pas ou peu d'historique de prêt.

Afin d'éviter des pertes importantes, l'entreprise souhaite mettre en oeuvre un outil d'évalutation du risque de crédit associé à un prospect avant une éventuelle signature.

Cet outil prendra la forme d'un "scoring crédit" et :
- calculera la probalité de défaut du prospect
- classifiera la demande de prêt : "accordé" ou "refusé"

# Imports de librairies

In [1]:
import pandas as pd # to manipulate the dataset with dataframes
import numpy as np # for statistics and calculations

import matplotlib.pyplot as plt # for data visualisation
%matplotlib inline
import matplotlib.ticker as mtick # for format axis in percent %
import seaborn as sns # for data visualisation, with different proporties
import missingno as ms # for visualisation of missing data
import plotly.express as px # for interactive graphs
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler, StandardScaler # for scaling

from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import KNNImputer, IterativeImputer, SimpleImputer # imputers

from sklearn import neighbors

from sklearn.metrics import mean_squared_error, r2_score # measures for imputation quality

from sklearn.decomposition import PCA # PCA decomposition

import statsmodels.api as sm
from statsmodels.formula.api import ols # for ANOVA
import scipy.stats as stats
from itertools import combinations

import ipywidgets as widgets

import os
import shutil
import sys

In [2]:
from myFunctions import bestDtype,testDtypes # for memory size management
from myFunctions import myDescribe # custom .Describe

# Import des datasets brutes

In [3]:
os.listdir("Projet+Mise+en+prod+-+home-credit-default-risk")

['application_test.csv',
 'application_train.csv',
 'bureau.csv',
 'bureau_balance.csv',
 'credit_card_balance.csv',
 'HomeCredit_columns_description.csv',
 'installments_payments.csv',
 'POS_CASH_balance.csv',
 'previous_application.csv',
 'sample_submission.csv']

In [4]:
earlyNumRows = None

In [5]:
if os.path.isdir("Projet+Mise+en+prod+-+home-credit-default-risk") :
    rawCsvNames=os.listdir("Projet+Mise+en+prod+-+home-credit-default-risk")
    rawCsvNames.remove('application_test.csv')
#     rawCsvNames.remove('HomeCredit_columns_description.csv')
#     rawCsvNames.remove('sample_submission.csv')
    rawDfNames=[
        "rawAppliDf",
        "rawBurDf",
        "rawBurBalDf",
        "rawCcBalDf",
        "rawColDescripDf",
        "rawInstalPaymDf",
        "rawPosDf",
        "rawPrevAppDf",
        "rawSampleSubmi"
    ]
    rawDfDict={}

    for dfName,csvName in zip(rawDfNames,rawCsvNames) :
        vars()[dfName]=pd.read_csv(
            "Projet+Mise+en+prod+-+home-credit-default-risk/"+csvName,
            encoding_errors="ignore",
            nrows=earlyNumRows
                       )
        rawDfDict[dfName]=vars()[dfName]
else : 
    print("attention le dossier de base contenant les .csv du projet n'est pas présent")

In [6]:
realRawDfNames=[ # create a list of the dataframes based on "real" datasets
    "rawAppliDf",
    "rawBurDf",
    "rawBurBalDf",
    "rawCcBalDf",
    "rawInstalPaymDf",
    "rawPosDf",
    "rawPrevAppDf"
]

realRawCsvNames=[ # same list but with corresponding csv files names
     'application_train.csv',
     'bureau.csv',
     'bureau_balance.csv',
     'credit_card_balance.csv',
     'installments_payments.csv',
     'POS_CASH_balance.csv',
     'previous_application.csv'
]

# Exploration

## Describe

In [7]:
myDescribe(rawDfDict)

Dropdown(description='Which dataframe :', options=('rawAppliDf', 'rawBurDf', 'rawBurBalDf', 'rawCcBalDf', 'raw…

Output()

Ce que l'on peut en retenir :
- la colonne `TARGET` est déséquilibrée. En effet, ce qui est assez logique, il y a peu (8%) de client avec des défaut de paiement.
- pour certaines features `DAYS...` nous renseignant sur un nombre de jours, le maximum est la valeur `365243`, ce qui est incohérent. Cette valeur représente surement $\infty$.
    *(exemple : dataframe `rawAppliDf` - feature `DAYS_EMPLOYED`)*
    

- certaines features qualitatives ne semblent comporter que 2 valeurs uniques. Celles-ci pourraient transformées en features numériques avec les valeurs 0 ou 1.
    *(exemple : dataframe `rawAppliDf` - feature `FLAG_OWN_CAR`)*

- sur le même principe, certaines features ont 3 valeurs uniques là il serait plus logique qu'elles en aient 2. la troisième valeur pourrait refléter une absence d'information, et donc passée en valeur manquante.
    *(exemple : dataframe `rawAppliDf` - feature `CODE_GENDER`)*
    


## Valeurs reflétant une absence d'information

Recherchons tout d'abord des valeurs représentant une absence d'information :

In [8]:
# # print qualitative unique values to find values that can be replace with NaN
# for dfName in realRawDfNames :
#     print(dfName)
#     print("-------------")
#     df=vars()[dfName].copy()
#     test=pd.Series(df.apply(lambda s : s.unique().tolist() \
#                                                            if ((s.dtype=="O")) \
#                                     else np.nan,axis=0

#                                                           ))
#     for i,v in test.items() :
#         print(i,":",v)
#     print("-------------")
#     del test, df

Les valeurs : `"XNA"`,  `'Unknown'`, `'not specified'`, `'Unknown type of loan'`, `'X'` peuvent être considérées comme des valeurs manquantes.

XXXXXXXXXXXX  `'XAP'` XXXXXXXXXXXXXXXXX

## Amender `HomeCredit_columns_description.csv` avec des informations utiles

In [9]:
rawColDescripDf.shape

(219, 5)

Il y a apparemment 219 features dans les différents datasets à notre disposition. Nous pouvons nous appuyer sur ce tableau `HomeCredit_columns_description.csv` pour présenter différentes informations sur nos features :

In [10]:
if not os.path.isdir("myCSVs") :
    os.mkdir("myCSVs")
if not os.path.isdir("myCSVs/processed") :
    os.mkdir("myCSVs/processed")
if not os.path.isdir("myCSVs/processed/columnsDescription") :
    os.mkdir("myCSVs/processed/columnsDescription")

In [11]:
%%time
# add informations in the columns description dataframe

if "proColDescripDf.csv" in os.listdir("myCSVs/processed/columnsDescription") :
    proColDescripDf=pd.read_csv("myCSVs/processed/columnsDescription/proColDescripDf.csv",index_col=0)
    colNamesWithListsInside = ["Uniques","otherNanValues","binValues"]
    for colName in colNamesWithListsInside :
        proColDescripDf.loc[proColDescripDf[colName].notna(),colName] \
        = \
        proColDescripDf.loc[proColDescripDf[colName].notna(),colName]\
        .apply(lambda x : x.strip("[]").replace("'","").split(", ") 
               if x != "365243.0" 
               else 365243
              )
else :
    proColDescripDf = rawColDescripDf.copy() # create copy


   
    # change "application_{train|test}.csv" for 'application_train.csv' in the columns description df : proColDescripDf
    proColDescripDf.loc[proColDescripDf["Table"]=="application_{train|test}.csv","Table"] \
    = 'application_train.csv'
    # correct proColDescripDf issues
    proColDescripDf=proColDescripDf.drop(columns="Unnamed: 0")
    proColDescripDf["Row"]=proColDescripDf["Row"].str.strip()
    proColDescripDf.loc[proColDescripDf["Row"]=="SK_BUREAU_ID","Row"]="SK_ID_BUREAU"
    proColDescripDf=proColDescripDf.loc[proColDescripDf["Row"]!="NFLAG_MICRO_CASH"]

    # create columns with informations


    for dfName,csvName in zip(realRawDfNames,realRawCsvNames) :  # iterate on dataframes and their csv files
        df=vars()[dfName].copy()
        mask = ( # in the columns description dataframe, filter on csvName and df's columns
            (proColDescripDf["Table"]==csvName)&
            (proColDescripDf["Row"].isin(df.columns))
        )
#         print(    proColDescripDf.loc[mask,"Row"].tolist()==df.columns.tolist()    )
        
        # TYPES
        
        # get columns dtypes
        proColDescripDf.loc[mask,"type"]=df.dtypes.values.tolist()
        
        # true or false - float columns values are all integers
        proColDescripDf.loc[mask,"float_real_int"]=(
            (
                (df.apply(lambda s : (s*10%10).max() \
                                  if s.dtype=="float64" \
                                  else np.nan,
                                  axis=0)
                )==0
            ).values.tolist())
        
        # lowest memory consumming dtype
        proColDescripDf.loc[mask,"bestType"] = proColDescripDf.loc[mask].apply(lambda r : bestDtype(r["Row"],df), axis=1)
                        
        # get percentage of missing values
        proColDescripDf.loc[mask,"NaN_rate"]=(df.isna().mean(axis=0)).values.tolist()

        # get unique values
        proColDescripDf.loc[mask,"Uniques"]=pd.Series(df.apply(lambda s : s.dropna().unique().tolist() \
                                                               if s.dtype=="O"
                                                               else np.nan,
                                                               axis=0
                                                      )).values

        # for concerned columns, add the new NaN value 365243 in a list 
        mask2 = (proColDescripDf["type"]=='int64')|(proColDescripDf["type"]=='float64')
        
        
        proColDescripDf.loc[mask&mask2,"otherNanValues"]=pd.Series(df.select_dtypes(['int64', 'float64'])\
                                                                    .apply(lambda s : 365243
                                                                              if s.max()==365243
                                                                              else np.nan,
                                                                              axis=0)).values
        
        
        
        del df,mask,mask2

    # list of new NaN values if columns have "XNA",  'Unknown', 'not specified', 'Unknown type of loan', 'X' within their values
    nanValuesList=["XNA",  'Unknown', 'not specified', 'Unknown type of loan','X']
    nanValuesList.append("XAP") # add "XAP"
    
    mask = proColDescripDf["otherNanValues"].isna()
    proColDescripDf.loc[mask,"otherNanValues"]=proColDescripDf.loc[mask,"Uniques"]\
    .apply(lambda lst : [nanVal for nanVal in nanValuesList if nanVal in lst] \
           if type(lst)==list
           else np.nan)\
    .apply(lambda lst : lst if (type(lst)==list and len(lst)>0) else np.nan)

    # true or false - columns have only 2 unique values
    proColDescripDf["bin"]=(
        proColDescripDf["Uniques"].apply(lambda l : len(l) if type(l)==list else np.nan) \
        - proColDescripDf["otherNanValues"].apply(lambda l : len(l) if type(l)==list else 0)
    )==2
    # store both values
    proColDescripDf.loc[(proColDescripDf["bin"]==True)&(proColDescripDf["otherNanValues"].isna()),"binValues"] = \
    proColDescripDf.loc[(proColDescripDf["bin"]==True)&(proColDescripDf["otherNanValues"].isna()),"Uniques"]
    
    
    proColDescripDf.loc[(proColDescripDf["bin"]==True)&(proColDescripDf["otherNanValues"].notna()),"binValues"] = \
    proColDescripDf.loc[(proColDescripDf["bin"]==True)&(proColDescripDf["otherNanValues"].notna()),["Uniques","otherNanValues"]]\
    .apply(lambda r : [elt for elt in r["Uniques"] if elt not in r["otherNanValues"]],
                                                axis=1)
    
    
    proColDescripDf.to_csv("myCSVs/processed/columnsDescription/proColDescripDf.csv")
    


CPU times: total: 0 ns
Wall time: 16.7 ms


In [12]:
proColDescripDf

,Table,Row,Description,Special,type,float_real_int,bestType,NaN_rate,Uniques,otherNanValues,bin,binValues
0,application_train.csv,SK_ID_CURR,ID of loan in our sample,NaN,int64,False,int32,0.000000,NaN,NaN,False,NaN
1,application_train.csv,TARGET,Target variable (1 - client with payment diffi...,NaN,int64,False,int8,0.000000,NaN,NaN,False,NaN
2,application_train.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN,object,False,category,0.000000,"[Cash loans, Revolving loans]",NaN,True,"[Cash loans, Revolving loans]"
3,application_train.csv,CODE_GENDER,Gender of the client,NaN,object,False,category,0.000000,"[M, F, XNA]",[XNA],True,"[M, F]"
4,application_train.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN,object,False,category,0.000000,"[N, Y]",NaN,True,"[N, Y]"
...,...,...,...,...,...,...,...,...,...,...,...,...
214,installments_payments.csv,NUM_INSTALMENT_NUMBER,On which installment we observe payment,NaN,int64,False,int16,0.000000,NaN,NaN,False,NaN
215,installments_payments.csv,DAYS_INSTALMENT,When the installment of previous credit was su...,time only relative to the application,float64,True,int16,0.000000,NaN,NaN,False,NaN
216,installments_payments.csv,DAYS_ENTRY_PAYMENT,When was the installments of previous credit p...,time only relative to the application,float64,True,float32,0.000214,NaN,NaN,False,NaN
217,installments_payments.csv,AMT_INSTALMENT,What was the prescribed installment amount of ...,NaN,float64,False,float64,0.000000,NaN,NaN,False,NaN


## Stocker des informations qui nous permettront d'améliorer nos imports

In [14]:
# dictionnary for dtypes
lowMemTypesDict = {
                    csvName : {
                                col : bestTyp for col,bestTyp in zip(
                                    proColDescripDf.loc[proColDescripDf["Table"]==csvName,"Row"].values.tolist(),
                                    proColDescripDf.loc[proColDescripDf["Table"]==csvName,"bestType"].values.tolist()
                                                                    )
                         

}
               for csvName in realRawCsvNames}



In [15]:
lowMemTypesDict

{'application_train.csv': {'SK_ID_CURR': 'int32',
  'TARGET': 'int8',
  'NAME_CONTRACT_TYPE': 'category',
  'CODE_GENDER': 'category',
  'FLAG_OWN_CAR': 'category',
  'FLAG_OWN_REALTY': 'category',
  'CNT_CHILDREN': 'int8',
  'AMT_INCOME_TOTAL': 'float64',
  'AMT_CREDIT': 'float32',
  'AMT_ANNUITY': 'float32',
  'AMT_GOODS_PRICE': 'float32',
  'NAME_TYPE_SUITE': 'category',
  'NAME_INCOME_TYPE': 'category',
  'NAME_EDUCATION_TYPE': 'category',
  'NAME_FAMILY_STATUS': 'category',
  'NAME_HOUSING_TYPE': 'category',
  'REGION_POPULATION_RELATIVE': 'float32',
  'DAYS_BIRTH': 'int16',
  'DAYS_EMPLOYED': 'int32',
  'DAYS_REGISTRATION': 'float32',
  'DAYS_ID_PUBLISH': 'int16',
  'OWN_CAR_AGE': 'float32',
  'FLAG_MOBIL': 'int8',
  'FLAG_EMP_PHONE': 'int8',
  'FLAG_WORK_PHONE': 'int8',
  'FLAG_CONT_MOBILE': 'int8',
  'FLAG_PHONE': 'int8',
  'FLAG_EMAIL': 'int8',
  'OCCUPATION_TYPE': 'category',
  'CNT_FAM_MEMBERS': 'float32',
  'REGION_RATING_CLIENT': 'int8',
  'REGION_RATING_CLIENT_W_CITY': 'i

In [16]:
# dictionnary for categorical columns nan values


catNanValuesDict = {
                    csvName : {
                                col : nanLst for col,nanLst in zip(
                                    
                                    proColDescripDf.loc[
                                        (
                                            (proColDescripDf["Table"]==csvName)&
                                            (proColDescripDf["otherNanValues"].notna())&
                                            (proColDescripDf["otherNanValues"]!=365243)
                                    ),
                                        "Row"].values.tolist(),
                                    
                                    proColDescripDf.loc[(
                                            (proColDescripDf["Table"]==csvName)&
                                            (proColDescripDf["otherNanValues"].notna())&
                                            (proColDescripDf["otherNanValues"]!=365243)
                                    ),
                                        "otherNanValues"].values.tolist()
                                )
                    }
    for csvName in realRawCsvNames
}



In [17]:
catNanValuesDict

{'application_train.csv': {'CODE_GENDER': ['XNA'],
  'NAME_FAMILY_STATUS': ['Unknown'],
  'ORGANIZATION_TYPE': ['XNA'],
  'FONDKAPREMONT_MODE': ['not specified']},
 'bureau.csv': {'CREDIT_TYPE': ['Unknown type of loan']},
 'bureau_balance.csv': {'STATUS': ['X']},
 'credit_card_balance.csv': {},
 'installments_payments.csv': {},
 'POS_CASH_balance.csv': {'NAME_CONTRACT_STATUS': ['XNA']},
 'previous_application.csv': {'NAME_CONTRACT_TYPE': ['XNA'],
  'NAME_CASH_LOAN_PURPOSE': ['XNA', 'XAP'],
  'NAME_PAYMENT_TYPE': ['XNA'],
  'CODE_REJECT_REASON': ['XNA', 'XAP'],
  'NAME_CLIENT_TYPE': ['XNA'],
  'NAME_GOODS_CATEGORY': ['XNA'],
  'NAME_PORTFOLIO': ['XNA'],
  'NAME_PRODUCT_TYPE': ['XNA'],
  'NAME_SELLER_INDUSTRY': ['XNA'],
  'NAME_YIELD_GROUP': ['XNA']}}

In [18]:
# dictionnary for numerical columns nan values


numNanValuesDict = {
                    csvName : {
                                col : nanLst for col,nanLst in zip(
                                    
                                    proColDescripDf.loc[
                                        (
                                            (proColDescripDf["Table"]==csvName)&
                                            (proColDescripDf["otherNanValues"]==365243)
                                    ),
                                        "Row"].values.tolist(),
                                    
                                    proColDescripDf.loc[(
                                            (proColDescripDf["Table"]==csvName)&
                                            (proColDescripDf["otherNanValues"]==365243)
                                    ),
                                        "otherNanValues"].values.tolist()
                                )
                    }
    for csvName in realRawCsvNames
}



In [19]:
numNanValuesDict

{'application_train.csv': {'DAYS_EMPLOYED': 365243},
 'bureau.csv': {},
 'bureau_balance.csv': {},
 'credit_card_balance.csv': {},
 'installments_payments.csv': {},
 'POS_CASH_balance.csv': {},
 'previous_application.csv': {'DAYS_FIRST_DRAWING': 365243,
  'DAYS_FIRST_DUE': 365243,
  'DAYS_LAST_DUE_1ST_VERSION': 365243,
  'DAYS_LAST_DUE': 365243,
  'DAYS_TERMINATION': 365243}}

In [20]:
# dictionnary for bin columns

booleanDict = {
                csvName : {
                            col : colVals for col,colVals in zip(
                                
                                proColDescripDf.loc[
                                    (
                                        (proColDescripDf["Table"]==csvName)&
                                        (proColDescripDf["bin"]==True)
                                    ),
                                    "Row"
                                ].values.tolist() 
                                ,
                                proColDescripDf.loc[
                                    (
                                        (proColDescripDf["Table"]==csvName)&
                                        (proColDescripDf["bin"]==True)
                                    ),
                                    "binValues"
                                ].values.tolist()
                            )
                }

               for csvName in realRawCsvNames}

In [21]:
booleanDict

{'application_train.csv': {'NAME_CONTRACT_TYPE': ['Cash loans',
   'Revolving loans'],
  'CODE_GENDER': ['M', 'F'],
  'FLAG_OWN_CAR': ['N', 'Y'],
  'FLAG_OWN_REALTY': ['Y', 'N'],
  'EMERGENCYSTATE_MODE': ['No', 'Yes']},
 'bureau.csv': {},
 'bureau_balance.csv': {},
 'credit_card_balance.csv': {},
 'installments_payments.csv': {},
 'POS_CASH_balance.csv': {},
 'previous_application.csv': {'FLAG_LAST_APPL_PER_CONTRACT': ['Y', 'N'],
  'NAME_PRODUCT_TYPE': ['x-sell', 'walk-in']}}

On peut maintenant attribuer les numéros 0 et 1 à ces valeurs :

In [22]:
booleanEncodeDict = {
    'Cash loans':1,'Revolving loans':0,
    'F':1,'M':0,
    'Y':1,'N':0,
    'Yes':1,'No':0,
    'walk-in':1,'x-sell':0
}

## Meilleurs dtypes - exemples

In [23]:
testDtypes ("AMT_INCOME_TOTAL",rawAppliDf)
bestDtype ("AMT_INCOME_TOTAL",rawAppliDf)

AMT_INCOME_TOTAL
dtype :  float64
NaN rate :  0.0
>0
---------
raw :  2460252  bytes (with  float64 )
signed :  2460252  bytes (with  float64 )
unsigned :  2460252  bytes (with  float64 )
float :  2460252  bytes (with  float64 )


dtype('float64')

In [24]:
testDtypes ("FLAG_MOBIL",rawAppliDf)
bestDtype ("FLAG_MOBIL",rawAppliDf)

FLAG_MOBIL
dtype :  int64
NaN rate :  0.0
>0
---------
raw :  2460252  bytes (with  int64 )
integer :  307675  bytes (with  int8 )
signed :  307675  bytes (with  int8 )
unsigned :  307675  bytes (with  uint8 )
float :  1230208  bytes (with  float32 )


dtype('int8')

In [25]:
testDtypes ("FLAG_OWN_CAR",rawAppliDf)
bestDtype ("FLAG_OWN_CAR",rawAppliDf)

FLAG_OWN_CAR
dtype :  object
NaN rate :  0.0
---------
raw :  17835802  bytes (with  object  )
Cat :  307899  bytes (with category)


'category'

In [26]:
testDtypes ("FLAG_OWN_CAR",rawAppliDf[["FLAG_OWN_CAR"]].applymap(lambda x : 0 if x=="N" else 1))
bestDtype ("FLAG_OWN_CAR",rawAppliDf[["FLAG_OWN_CAR"]].applymap(lambda x : 0 if x=="N" else 1))

FLAG_OWN_CAR
dtype :  int64
NaN rate :  0.0
>0
---------
raw :  2460252  bytes (with  int64 )
integer :  307675  bytes (with  int8 )
signed :  307675  bytes (with  int8 )
unsigned :  307675  bytes (with  uint8 )
float :  1230208  bytes (with  float32 )


dtype('int8')

In [27]:
testDtypes ("DEF_60_CNT_SOCIAL_CIRCLE",rawAppliDf)
bestDtype ("DEF_60_CNT_SOCIAL_CIRCLE",rawAppliDf)

DEF_60_CNT_SOCIAL_CIRCLE
dtype :  float64
NaN rate :  0.0033202064316398437
>0
---------
raw :  2460252  bytes (with  float64 )
signed :  2460252  bytes (with  float64 )
unsigned :  2460252  bytes (with  float64 )
float :  1230208  bytes (with  float32 )


dtype('float32')

In [28]:
testDtypes ("DAYS_REGISTRATION",rawAppliDf)
bestDtype ("DAYS_REGISTRATION",rawAppliDf)

DAYS_REGISTRATION
dtype :  float64
NaN rate :  0.0
>0
---------
raw :  2460252  bytes (with  float64 )
signed :  2460252  bytes (with  float64 )
unsigned :  2460252  bytes (with  float64 )
float :  1230208  bytes (with  float32 )


dtype('float32')

In [29]:
testDtypes ("CODE_GENDER",rawAppliDf)
bestDtype ("CODE_GENDER",rawAppliDf)

CODE_GENDER
dtype :  object
NaN rate :  0.0
---------
raw :  17835810  bytes (with  object  )
Cat :  307959  bytes (with category)


'category'

In [30]:
testDtypes ("HOUSETYPE_MODE",rawAppliDf)
bestDtype ("CODE_GENDER",rawAppliDf)

HOUSETYPE_MODE
dtype :  object
NaN rate :  0.50176091261776
---------
raw :  15818860  bytes (with  object  )
Cat :  307998  bytes (with category)


'category'

In [31]:
proColDescripDf.bestType.value_counts()

bestType
float32     83
int8        43
category    38
float64     25
int32       16
int16       11
uint8        2
Name: count, dtype: int64

## `rawAppliDf`

## `rawBurDf`

## `rawBurBalDf`

## `rawCcBalDf`

## `rawInstalPaymDf`

## `rawPosDf`

## `rawPrevAppDf`

## Imports optimisés des datasets

In [32]:
if not os.path.isdir("myCSVs") :
    os.mkdir("myCSVs")
if not os.path.isdir("myCSVs/processed") :
    os.mkdir("myCSVs/processed")
if not os.path.isdir("myCSVs/processed/lowMem") :
    os.mkdir("myCSVs/processed/lowMem")

In [33]:
lowMemDfNames=[ # create a list of df names for optimized imports
    "lowMemAppliDf",
    "lowMemBurDf",
    "lowMemBurBalDf",
    "lowMemCcBalDf",
    "lowMemInstalPaymDf",
    "lowMemPosDf",
    "lowMemPrevAppDf"
]

In [34]:
def importsEnhanced(
    dfName,
    csvName,
    dtypes=lowMemTypesDict,
    catNans=catNanValuesDict,
    numNans=numNanValuesDict,
    boolCols=booleanDict,
    boolEncodeKeys=booleanEncodeDict
) :

    df=pd.read_csv(
        "Projet+Mise+en+prod+-+home-credit-default-risk/"+csvName,
        encoding_errors="ignore",
        dtype=dtypes[csvName],
        na_values=catNans[csvName]
                   )

    for colName in numNans[csvName].keys() :
        df[colName]=df[colName].replace(numNans[csvName][colName],np.nan)
        df[colName]=df[colName].astype(bestDtype(colName,df))

    for colName in boolCols[csvName].keys() :
        df[colName]=pd.to_numeric(df[colName].apply(lambda x : boolEncodeKeys[x]))
        df[colName]=df[colName].astype(bestDtype(colName,df))
        
    return df

In [35]:
%%time
enhancedDfDict={}
for dfName,csvName in zip(lowMemDfNames,realRawCsvNames) :
    
    if dfName+".csv" in os.listdir("myCSVs/processed/lowMem") :
        df=pd.read_csv("myCSVs/processed/lowMem/"+dfName+".csv",index_col=0)
        df=df.astype({col : bestDtype(col,df) for col in df.columns})
        vars()[dfName]=enhancedDfDict[dfName]=df
        del df

    else :
        vars()[dfName]=enhancedDfDict[dfName]=importsEnhanced(dfName,csvName)
        vars()[dfName].to_csv("myCSVs/processed/lowMem/"+dfName+".csv")

CPU times: total: 18.3 s
Wall time: 1min 42s


Nous pouvons comparer l'espace mémoire :

In [36]:
if not os.path.isdir("myCSVs/processed/memorySizes") :
    os.mkdir("myCSVs/processed/memorySizes")

if "memorySizesRawVSEnhanced.csv" in os.listdir("myCSVs/processed/memorySizes") :
    memorySizesRawVSEnhanced=pd.read_csv("myCSVs/processed/memorySizes/memorySizesRawVSEnhanced.csv",index_col=0)
else :
    memorySizesRawVSEnhanced = pd.DataFrame()

    memorySizesRawVSEnhanced.index=realRawCsvNames

    memorySizesRawVSEnhanced["rawImports"]=[sys.getsizeof(rawDfDict[dfName]) for dfName in realRawDfNames]
    memorySizesRawVSEnhanced["enhancedImports"]=[sys.getsizeof(enhancedDfDict[dfName]) for dfName in lowMemDfNames]

    memorySizesRawVSEnhanced["difference"]=\
    ((memorySizesRawVSEnhanced["enhancedImports"]/memorySizesRawVSEnhanced["rawImports"]-1)*100).astype(int).astype(str)+" %"

    memorySizesRawVSEnhanced.to_csv("myCSVs/processed/memorySizes/memorySizesRawVSEnhanced.csv")

In [37]:
memorySizesRawVSEnhanced

,rawImports,enhancedImports,difference
application_train.csv,562761965,105489612,-81 %
bureau.csv,536987086,144182495,-73 %
bureau_balance.csv,2020194614,382199688,-81 %
credit_card_balance.csv,918225104,48974088,-94 %
installments_payments.csv,870745828,557821473,-35 %
POS_CASH_balance.csv,1192493276,300041606,-74 %
previous_application.csv,1992956783,205449783,-89 %


In [48]:
lowMemInstalPaymDf.describe(include=None)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
count,1.360540e+07,1.360540e+07,1.360540e+07,1.360540e+07,1.360540e+07,1.360250e+07,1.360540e+07,1.360250e+07
mean,1.903365e+06,2.784449e+05,8.566373e-01,1.887090e+01,-1.042270e+03,-1.051114e+03,1.705091e+04,1.723822e+04
std,5.362029e+05,1.027183e+05,1.035216e+00,2.666407e+01,8.009463e+02,8.005859e+02,5.057025e+04,5.473578e+04
min,1.000001e+06,1.000010e+05,0.000000e+00,1.000000e+00,-2.922000e+03,-4.921000e+03,0.000000e+00,0.000000e+00
25%,1.434191e+06,1.896390e+05,0.000000e+00,4.000000e+00,-1.654000e+03,-1.662000e+03,4.226085e+03,3.398265e+03
50%,1.896520e+06,2.786850e+05,1.000000e+00,8.000000e+00,-8.180000e+02,-8.270000e+02,8.884080e+03,8.125515e+03
75%,2.369094e+06,3.675300e+05,1.000000e+00,1.900000e+01,-3.610000e+02,-3.700000e+02,1.671021e+04,1.610842e+04
max,2.843499e+06,4.562550e+05,1.780000e+02,2.770000e+02,-1.000000e+00,-1.000000e+00,3.771488e+06,3.771488e+06


In [44]:
myDescribe(enhancedDfDict)

Dropdown(description='Which dataframe :', options=('lowMemAppliDf', 'lowMemBurDf', 'lowMemBurBalDf', 'lowMemCc…

Output()

In [52]:
lowMemBurDf[lowMemBurDf.CREDIT_ACTIVE=="Active"].SK_ID_CURR.nunique()

251815

In [53]:
lowMemBurDf[lowMemBurDf.CREDIT_ACTIVE=="Closed"].SK_ID_CURR.nunique()

267925

In [54]:
lowMemBurDf.SK_ID_CURR.nunique()

305811

In [107]:
burTest = lowMemBurDf[["SK_ID_CURR","CREDIT_ACTIVE","DAYS_CREDIT"]]

burTest,catBurTestCols = one_hot_encoder(burTest)

numAgreg  ={"DAYS_CREDIT" : ["min","max","mean"]}
catAgreg = {col : ["mean"] for col in catBurTestCols}

burTestAgg=burTest.groupby("SK_ID_CURR").agg({**numAgreg,**catAgreg})
burTestAgg
# burTestAgg.columns=pd.Index(["BURO_"+col[0]+"_"+col[1].upper() for col in burTestAgg.columns])

# # active
# active = burTest[burTest.CREDIT_ACTIVE_Active==1]

# activeAgg=active.groupby("SK_ID_CURR").agg({**numAgreg})

# activeAgg.columns=pd.Index(["ACTIVE_"+col[0]+"_"+col[1].upper() for col in activeAgg.columns])

# burTestAgg.merge(activeAgg,how="left",on="SK_ID_CURR")

DAYS_CREDIT                    CREDIT_ACTIVE_Active  \
                   min   max         mean                 mean   
SK_ID_CURR                                                       
100001           -1572   -49  -735.000000             0.428571   
100002           -1437  -103  -874.000000             0.250000   
100003           -2586  -606 -1400.750000             0.250000   
100004           -1326  -408  -867.000000             0.000000   
100005            -373   -62  -190.666667             0.666667   
...                ...   ...          ...                  ...   
456249           -2713  -483 -1667.076923             0.153846   
456250           -1002  -760  -862.000000             0.666667   
456253            -919  -713  -867.500000             0.500000   
456254           -1104 -1104 -1104.000000             0.000000   
456255           -2337  -363 -1089.454545             0.454545   

           CREDIT_ACTIVE_Bad debt CREDIT_ACTIVE_Closed CREDIT_ACTIVE_Sold  \
                             mean                 mean               mean   
SK_ID_CURR                                                                  
100001                        0.0             0.571429                0.0   
100002                        0.0             0.750000                0.0   
100003                        0.0             0.750000                0.0   
100004                        0.0             1.000000                0.0   
100005                        0.0             0.333333                0.0   
...                           ...                  ...                ...   
456249                        0.0             0.846154                0.0   
456250                        0.0             0.333333                0.0   
456253                        0.0             0.500000                0.0   
456254                        0.0             1.000000                0.0   
456255                        0.0             0.545455                0.0   

           CREDIT_ACTIVE_nan  
                        mean  
SK_ID_CURR                    
100001                   0.0  
100002                   0.0  
100003                   0.0  
100004                   0.0  
100005                   0.0  
...                      ...  
456249                   0.0  
456250                   0.0  
456253                   0.0  
456254                   0.0  
456255                   0.0  

[305811 rows x 8 columns]

## Aggrégation des datasets

### Encodage des features catégorielles

In [90]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'category']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category, dtype = "int8")
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [91]:
encodedDfNames=[ # create a list of one hot encoded df names
    "encodedAppliDf",
    "encodedBurDf",
    "encodedBurBalDf",
    "encodedCcBalDf",
    "encodedInstalPaymDf",
    "encodedPosDf",
    "encodedPrevAppDf"
]

catEncodedColNames=[ # create a list of list names for new encoded columns names
    "encodedColNamesAppli",
    "encodedColNamesBur",
    "encodedColNamesBurBal",
    "encodedColNamesCcBal",
    "encodedColNamesInstalPaym",
    "encodedColNamesPos",
    "encodedColNamesPrevApp"
]

In [92]:
for encodedDfName, catEncodedColName, dfName in zip(encodedDfNames,catEncodedColNames,lowMemDfNames) :
    vars()[encodedDfName],vars()[catEncodedColName] = one_hot_encoder(vars()[dfName])

### Fonction d'aggrégation

In [127]:
def aggregationFunc (df, 
                     newColsText, 
                     groupByColName,
                     aggDict,
                    ) :
    '''
    Aggretates the values in a NUMERIC (already encoded) dataframe
    
    parameters
    ---------
    df - dataframe : dataframe used to calculate aggregations on
    newColsText - string : text added in new features names
    groupByColName - string : name of the columns to perform pandas.groupby on
    aggDict - dict : dictionnary with :
                        - keys - string : columns names to aggregate on
                        - values - string or list : aggregation function(s) for each columns
    
    return 
    ------
    aggDf - dataframe : the same dataframe with :
                        - the groupBy variable in first column
                        - for each column name in aggDict, as many columns as functions in the corresponding aggDict value
    
    
    '''
    
    # dealing with functions names not in lists
    for colName,funcs in aggDict.items() :
        if type(funcs)==str :
            aggDict[colName]=[funcs]
    
    # perform groupby
    aggDf = df.groupby(groupByColName).agg(aggDict)
    
    aggDf.columns = pd.Index([col[0]+"_Agg"+col[1].capitalize() for col in aggDf.columns])
    
    return aggDf
    


In [185]:
burTestCols=["SK_ID_CURR","DAYS_CREDIT"]+[col for col in encodedBurDf.columns if "ACTIVE" in col]
burTest = encodedBurDf[burTestCols]
burTest

,SK_ID_CURR,DAYS_CREDIT,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_ACTIVE_nan
0,215354,-497,0,0,1,0,0
1,215354,-208,1,0,0,0,0
2,215354,-203,1,0,0,0,0
3,215354,-203,1,0,0,0,0
4,215354,-629,1,0,0,0,0
...,...,...,...,...,...,...,...
1716423,259355,-44,1,0,0,0,0
1716424,100044,-2648,0,0,1,0,0
1716425,100044,-1809,0,0,1,0,0
1716426,246829,-1878,0,0,1,0,0


In [187]:
testAggDict = {"DAYS_CREDIT" : ["min","max","sum","mean"]} \
|{col : ["mean"] for col in [col for col in encodedBurDf.columns if "ACTIVE" in col]}


burAggTest=aggregationFunc (df=burTest, 
                 newColsText="TEST", 
                 groupByColName="SK_ID_CURR",
                 aggDict=testAggDict,
                )
burAggTest

,DAYS_CREDIT_AggMin,DAYS_CREDIT_AggMax,DAYS_CREDIT_AggSum,DAYS_CREDIT_AggMean,CREDIT_ACTIVE_Active_AggMean,CREDIT_ACTIVE_Bad debt_AggMean,CREDIT_ACTIVE_Closed_AggMean,CREDIT_ACTIVE_Sold_AggMean,CREDIT_ACTIVE_nan_AggMean
SK_ID_CURR,,,,,,,,,
100001,-1572,-49,-5145,-735.000000,0.428571,0.0,0.571429,0.0,0.0
100002,-1437,-103,-6992,-874.000000,0.250000,0.0,0.750000,0.0,0.0
100003,-2586,-606,-5603,-1400.750000,0.250000,0.0,0.750000,0.0,0.0
100004,-1326,-408,-1734,-867.000000,0.000000,0.0,1.000000,0.0,0.0
100005,-373,-62,-572,-190.666667,0.666667,0.0,0.333333,0.0,0.0
...,...,...,...,...,...,...,...,...,...
456249,-2713,-483,-21672,-1667.076923,0.153846,0.0,0.846154,0.0,0.0
456250,-1002,-760,-2586,-862.000000,0.666667,0.0,0.333333,0.0,0.0
456253,-919,-713,-3470,-867.500000,0.500000,0.0,0.500000,0.0,0.0


In [183]:
appliTest = encodedAppliDf[["SK_ID_CURR","TARGET","NAME_CONTRACT_TYPE"]].copy()
appliTest

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE
0,100002,1,1
1,100003,0,1
2,100004,0,0
3,100006,0,1
4,100007,0,1
...,...,...,...
307506,456251,0,1
307507,456252,0,1
307508,456253,0,1
307509,456254,1,1


In [179]:
appliTest = encodedAppliDf[["SK_ID_CURR","TARGET","NAME_CONTRACT_TYPE"]].copy()

appliTest["SK_ID_CURR"]=appliTest.SK_ID_CURR.astype("int64")

appliTest=appliTest.merge(burAggTest,how="left",on="SK_ID_CURR")
appliTest["SK_ID_CURR"]=appliTest.SK_ID_CURR.astype(bestDtype("SK_ID_CURR",appliTest))
appliTest

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,DAYS_CREDIT_AggMin,DAYS_CREDIT_AggMax,DAYS_CREDIT_AggSum,DAYS_CREDIT_AggMean,CREDIT_ACTIVE_Active_AggMean,CREDIT_ACTIVE_Bad debt_AggMean,CREDIT_ACTIVE_Closed_AggMean,CREDIT_ACTIVE_Sold_AggMean,CREDIT_ACTIVE_nan_AggMean
0,100002,1,1,-1437.0,-103.0,-6992.0,-874.000000,0.250000,0.0,0.750000,0.0,0.0
1,100003,0,1,-2586.0,-606.0,-5603.0,-1400.750000,0.250000,0.0,0.750000,0.0,0.0
2,100004,0,0,-1326.0,-408.0,-1734.0,-867.000000,0.000000,0.0,1.000000,0.0,0.0
3,100006,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,1,-1149.0,-1149.0,-1149.0,-1149.000000,0.000000,0.0,1.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,1,-919.0,-713.0,-3470.0,-867.500000,0.500000,0.0,0.500000,0.0,0.0
307509,456254,1,1,-1104.0,-1104.0,-1104.0,-1104.000000,0.000000,0.0,1.000000,0.0,0.0


In [131]:
encodedBurDf.columns

Index(['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE',
       'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE',
       'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT',
       'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY', 'CREDIT_ACTIVE_Active', 'CREDIT_ACTIVE_Bad debt',
       'CREDIT_ACTIVE_Closed', 'CREDIT_ACTIVE_Sold', 'CREDIT_ACTIVE_nan',
       'CREDIT_CURRENCY_currency 1', 'CREDIT_CURRENCY_currency 2',
       'CREDIT_CURRENCY_currency 3', 'CREDIT_CURRENCY_currency 4',
       'CREDIT_CURRENCY_nan', 'CREDIT_TYPE_Another type of loan',
       'CREDIT_TYPE_Car loan', 'CREDIT_TYPE_Cash loan (non-earmarked)',
       'CREDIT_TYPE_Consumer credit', 'CREDIT_TYPE_Credit card',
       'CREDIT_TYPE_Interbank credit',
       'CREDIT_TYPE_Loan for business development',
       'CREDIT_TYPE_Loan for purchase of shares (margin lending)',
       'CREDIT_TYPE_Loan for the purchase of equipment',
  

In [166]:
burTest = lowMemBurDf[["SK_ID_CURR","CREDIT_ACTIVE","DAYS_CREDIT"]]

burTest,catBurTestCols = one_hot_encoder(burTest)

numAgreg  ={"DAYS_CREDIT" : ["sum","size","count"]}
catAgreg = {col : ["mean"] for col in catBurTestCols}

burTestAgg=burTest.groupby("SK_ID_CURR").agg({**numAgreg,**catAgreg})
burTestAgg
burTestAgg.columns=pd.Index(["BURO_"+col[0]+"_"+col[1].upper() for col in burTestAgg.columns])

# active
active = burTest[burTest.CREDIT_ACTIVE_Active==1]

activeAgg=active.groupby("SK_ID_CURR").agg({**numAgreg})

activeAgg.columns=pd.Index(["ACTIVE_"+col[0]+"_"+col[1].upper() for col in activeAgg.columns])

burTestAgg.merge(activeAgg,how="left",on="SK_ID_CURR")


,BURO_DAYS_CREDIT_SUM,BURO_DAYS_CREDIT_SIZE,BURO_DAYS_CREDIT_COUNT,BURO_CREDIT_ACTIVE_Active_MEAN,BURO_CREDIT_ACTIVE_Bad debt_MEAN,BURO_CREDIT_ACTIVE_Closed_MEAN,BURO_CREDIT_ACTIVE_Sold_MEAN,BURO_CREDIT_ACTIVE_nan_MEAN,ACTIVE_DAYS_CREDIT_SUM,ACTIVE_DAYS_CREDIT_SIZE,ACTIVE_DAYS_CREDIT_COUNT
SK_ID_CURR,,,,,,,,,,,
100001,-5145,7,7,0.428571,0.0,0.571429,0.0,0.0,-928.0,3.0,3.0
100002,-6992,8,8,0.250000,0.0,0.750000,0.0,0.0,-1145.0,2.0,2.0
100003,-5603,4,4,0.250000,0.0,0.750000,0.0,0.0,-606.0,1.0,1.0
100004,-1734,2,2,0.000000,0.0,1.000000,0.0,0.0,NaN,NaN,NaN
100005,-572,3,3,0.666667,0.0,0.333333,0.0,0.0,-199.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
456249,-21672,13,13,0.153846,0.0,0.846154,0.0,0.0,-3095.0,2.0,2.0
456250,-2586,3,3,0.666667,0.0,0.333333,0.0,0.0,-1584.0,2.0,2.0
456253,-3470,4,4,0.500000,0.0,0.500000,0.0,0.0,-1632.0,2.0,2.0
